# Tests for quering local PBF osm files

guide for using the filter https://pypi.org/project/esy-osmfilter/


## imports

In [ ]:
import configparser, contextlib
import os, sys

from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter 
from esy.osmfilter import Node, Way, Relation

import urllib.request


## Get file

In [ ]:
c = 'liechtenstein-latest'
city_name = 'berlin-latest'

PBF_inputfile = os.path.join(os.getcwd(), f'test-osmfilter/{city_name}.osm.pbf')
JSON_outputfile = os.path.join(os.getcwd(), f'test-osmfilter/{city_name}.json')

PBF_inputfile

In [ ]:
if not os.path.exists(f'test-osmfilter/{city_name}.osm.pbf'):
    filename, headers = urllib.request.urlretrieve(
        f'https://download.geofabrik.de/europe/{city_name}.osm.pbf',
        filename=f'test-osmfilter/{city_name}.osm.pbf')
# PBF_inputfile = filename

## Filter main data

In [ ]:
prefilter   = {Node: {}, Way: {"man_made":["pipeline",],}, Relation: {}}
blackfilter = [("pipeline","substation"),]
whitefilter = [(('man_made','pipeline'),('waterway','drain')),] #[(("waterway","drain"),("name","Wäschgräble")), ]

In [ ]:
>>> [Data,_]=run_filter('noname',
...                     PBF_inputfile, 
...                     JSON_outputfile, 
...                     prefilter,
...                     whitefilter, 
...                     blackfilter, 
...                     NewPreFilterData=True, 
...                     CreateElements=False, 
...                     LoadElements=False,
...                     verbose=True)

In [ ]:
>>> len(Data['Node'])

In [ ]:
>>> len(Data['Relation'])

In [ ]:
>>> len(Data['Way'])

In [ ]:
Data['Way']

## Filter out (blackfilter)

In [ ]:
>>> [_,Elements]=run_filter('funny-waterway-pipelines',
...                            PBF_inputfile, 
...                            JSON_outputfile, 
...                            prefilter,
...                            whitefilter, 
...                            blackfilter, 
...                            NewPreFilterData=False, 
...                            CreateElements=True, 
...                            LoadElements=False,
...                            verbose=True)
>>> len(Elements['funny-waterway-pipelines'])
0
>>> len(Elements['funny-waterway-pipelines']['Relation'])
0
>>> len(Elements['funny-waterway-pipelines']['Way'])

In [ ]:
Elements

## save geojson

In [ ]:
>>> from esy.osmfilter import export_geojson

# jsontype='Line' 'Point' 

>>> export_geojson(Elements['funny-waterway-pipelines']['Way'],Data,
... filename=f'test-osmfilter/{city_name}.geojson',jsontype='Line')

## open geosjon

In [ ]:
import geopandas as gpd

elements = gpd.read_file(f'test-osmfilter/{city_name}.geojson')
elements['geometry'][0]

In [ ]:
elements.describe()

# shapely, create geometry from geojson

In [ ]:
import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize

# test

In [ ]:
green = {'landuse':['grass','forest','orchard','allotments','cementery','flowerbed', 'meadow','greenfield', 'recreation_ground','village_green'],
         'leisure':['park','playground','garden'],
         'natural':['heath','shrubbery','wood','grassland'],}
amenit = {'amenity':['bbq'],}

cycle = {'highway':['pedestrian','cycleway']}


prefilter   = {Node: {}, Way: cycle, Relation: {}}


[Data,_]=run_filter('noname',
                    PBF_inputfile, 
                    JSON_outputfile, 
                    prefilter,
                    whitefilter, 
                    blackfilter, 
                    NewPreFilterData=True, 
                    CreateElements=False, 
                    LoadElements=False,
                    verbose=True)

In [ ]:
ways = len(Data['Way'])
rels = len(Data['Relation'])
nodes = len(Data['Node'])
           
print(f"{ways} Ways")
print(f"{rels} Rels")
print(f"{nodes} Nodes")

In [ ]:
Data['Way']

In [ ]:
# jsontype='Line' 'Point' 

export_geojson(Data['Way'],Data,
filename=f'test-osmfilter/{city_name}.geojson',jsontype='Line')

## Create Shapely geomtry

In [ ]:
elements = gpd.read_file(f'test-osmfilter/{city_name}.geojson')
elements['geometry'][3]

In [ ]:
polygonize(elements['geometry'][1])

In [ ]:
lis = []
for l in elements['geometry']:
    lis.append(l)

In [ ]:
lis[2]

In [ ]:
type(lis[2])

In [ ]:
# converting to multi points

lis_p = geometry.MultiPoint(lis)


In [ ]:
# converting to multi lines

merged = linemerge([*lis])  # merge LineStrings
borders = unary_union(merged)  # linestrings to a MultiLineString
lines = geometry.MultiLineString(borders)

In [ ]:
lines

In [ ]:
# converting to mulpolygon

merged = linemerge([*lis])  # merge LineStrings
borders = unary_union(merged)  # linestrings to a MultiLineString
polygons = list(polygonize(borders))
parks = geometry.MultiPolygon(polygons)

In [ ]:
parks.geoms[3]

In [ ]:
type(parks)

## Create Geojson

In [ ]:
import folium
from shapely.geometry import mapping


In [ ]:
mapping(lines)

## example map in folium

In [ ]:
map_test = folium.Map(width=750,location=[52.5205,13.3988],height=500,tiles="openstreetmap",   zoom_start=11)
folium.GeoJson(mapping(lines)).add_to(map_test)

In [ ]:
map_test

# Final

## Imports

In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import configparser, contextlib
import os, sys

# from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter 
from esy.osmfilter import Node, Way, Relation
from esy.osmfilter import export_geojson


import urllib.request

import geopandas as gpd

import folium

import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize

from shapely.geometry import mapping
from geojson import dump

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from livablestreets.query_names_detailed import master_query, master_query_complex, master_query_negative
from livablestreets.query_osmfilter import data_from_pbf

In [36]:
# df = master_query_complex()
df = master_query_negative()

df['geomtype'].iloc[0]

'Way'

In [37]:
df

,query_string,name,distance,geomtype,jsontype,shapelytype,category,whitefilter,blackfilter
street_motorway,{'highway': ['motorway']},street_motorway,300,Way,Line,lineString,negative,"[((, ), (, ))]","[(, )]"
street_primary,{'highway': ['primary']},street_primary,200,Way,Line,lineString,negative,"[((, ), (, ))]","[(, )]"
street_secondary,{'highway': ['secondary']},street_secondary,100,Way,Line,lineString,negative,"[((, ), (, ))]","[(, )]"
street_terteary,{'highway': ['terteary']},street_terteary,50,Way,Line,lineString,negative,"[((, ), (, ))]","[(, )]"


In [38]:
new = df.iloc[21:22]

In [39]:
new

,query_string,name,distance,geomtype,jsontype,shapelytype,category,whitefilter,blackfilter


In [40]:
df.index[0]

'street_motorway'

In [41]:
df['query_string'].iloc[0]

{'highway': ['motorway']}

## get file

https://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf

In [42]:
def get_pbf_json(country= 'germany',city = 'berlin'):
    #Download PBF file from geofabrik
    #example url: https://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf
    #Returns the file paths for use at Run_filter

    city_name = f'{city}-latest'

    if not os.path.exists(f'../livablestreets/data/{city}/{city_name}.osm.pbf'):
        filename, headers = urllib.request.urlretrieve(
            f'https://download.geofabrik.de/europe/{country}/{city_name}.osm.pbf',
            filename=f'../livablestreets/data/{city}/{city_name}.osm.pbf')
        PBF_inputfile = filename

    #gets the path for PBF
    PBF_inputfile = os.path.join(os.getcwd(), f'../livablestreets/data/{city}/{city_name}.osm.pbf')
    JSON_outputfile = os.path.join(os.getcwd(), f'../livablestreets/data/{city}/{city_name}.json')


    return PBF_inputfile, JSON_outputfile


In [43]:
PBF_inputfile, JSON_outputfile = get_pbf_json('germany','berlin')

In [44]:
PBF_inputfile

'/home/laia/code/Laiagdla/livablestreets/notebooks/../livablestreets/data/berlin/berlin-latest.osm.pbf'

In [45]:
JSON_outputfile

'/home/laia/code/Laiagdla/livablestreets/notebooks/../livablestreets/data/berlin/berlin-latest.json'

## execute filter

In [52]:
def shapely_format(lis,shapelytype):
    if shapelytype == 'Point':
        # converting to multi points
        shapely_geom = geometry.MultiPoint(lis)

    if shapelytype == 'lineString':
    # converting to multi lines
        merged = linemerge([*lis])  # merge LineStrings
        borders = unary_union(merged)  # linestrings to a MultiLineString
        shapely_geom = geometry.MultiLineString(lis)      

    if shapelytype == 'multiPolygon':
    #converting to multipolygon
        merged = linemerge([*lis])  # merge LineStrings
        borders = unary_union(merged)  # linestrings to a MultiLineString
        polygons = list(polygonize(borders))
        shapely_geom = geometry.MultiPolygon(lis)
        
    return shapely_geom



def run_filter(query_df, country= 'germany' , city_name= 'berlin'):
    # run filter from PBF file
    # get file paths
    PBF_inputfile, JSON_outputfile = get_pbf_json(country,city_name)

    # loops over dataframe of queries from query_names_detailed
    for index, row in query_df.iterrows():
        filter_name = index
        string = row['query_string']
        geometry = row['geomtype']

        jsontype = row['jsontype']
        category = row['category']
        shapelytype = row['shapelytype']

        if not os.path.exists(f'../livablestreets/data/{city_name}/Features/df-{category}-{filter_name}.geojson'):
            # initilize filter
            print(f'Filtering fetures: {filter_name} - {category}')

            # calls filter from query_osmfilter.py
            Data = data_from_pbf(filter_name, string, PBF_inputfile, JSON_outputfile)
            export_geojson(Data[geometry],Data,filename=f'../livablestreets/data/{city_name}/Features/df-{category}-{filter_name}.geojson',jsontype=jsontype)

        if not os.path.exists(f'../livablestreets/data/{city_name}/Features/shapes-{category}-{filter_name}.geojson'):
            print(f'Exporting shapes of: {filter_name} - {category} as: {geometry}')
            elements = gpd.read_file(f'../livablestreets/data/{city_name}/Features/df-{category}-{filter_name}.geojson')

            # create list of features
            lis = []
            for l in elements['geometry']:
                lis.append(l)

            shapely_geo = shapely_format(lis,shapelytype)

            gjson = mapping(shapely_geo)
            with open(f'../livablestreets/data/{city_name}/Features/shapes-{category}-{filter_name}.geojson', 'w') as f:
                dump(gjson, f)

#             df = pd.DataFrame(gjson["coordinates"], columns=['lng','lat'])
#             print(df.iloc[1])
            # gdf = gjson.explode(index_parts=True)
            # print(gdf['coordinates'])
#             df.to_csv(f'../livablestreets/data/{city_name}/Features/shapes-{category}-{filter_name}.csv', index=False)



In [53]:
run_filter(df,'germany','berlin')

INFO:esy.osmfilter.pre_filter:PreFilter OSM GAS DATA
INFO:esy.osmfilter.pre_filter:InputFile     : /home/laia/code/Laiagdla/livablestreets/notebooks/../livablestreets/data/berlin/berlin-latest.osm.pbf
INFO:esy.osmfilter.pre_filter:Size          : 70008           kbyte
INFO:esy.osmfilter.pre_filter:Estimated Time: 10.00           s
INFO:esy.osmfilter.pre_filter:=============================


Filtering fetures: street_secondary - negative


INFO:esy.osmfilter.pre_filter:0.5
INFO:esy.osmfilter.pre_filter:1
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilt

INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfi

INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfi

...

Exporting shapes of: street_secondary - negative as: Way


INFO:esy.osmfilter.pre_filter:PreFilter OSM GAS DATA
INFO:esy.osmfilter.pre_filter:InputFile     : /home/laia/code/Laiagdla/livablestreets/notebooks/../livablestreets/data/berlin/berlin-latest.osm.pbf
INFO:esy.osmfilter.pre_filter:Size          : 70008           kbyte
INFO:esy.osmfilter.pre_filter:Estimated Time: 10.00           s
INFO:esy.osmfilter.pre_filter:=============================


Filtering fetures: street_terteary - negative


INFO:esy.osmfilter.pre_filter:0.5
INFO:esy.osmfilter.pre_filter:1
INFO:esy.osmfilter.pre_filter:1.3
INFO:esy.osmfilter.pre_filter:2
INFO:esy.osmfilter.pre_filter:3
INFO:esy.osmfilter.pre_filter:Outputfile    : /home/laia/code/Laiagdla/livablestreets/notebooks/../livablestreets/data/berlin/berlin-latest.json
Size          : 0               kbyte 
Time Elapsed  : 20.25           s




Exporting shapes of: street_terteary - negative as: Way


In [54]:
Data

NameError: name 'Data' is not defined

# Cheking the data

In [60]:
import folium
from shapely.geometry import mapping
from folium.plugins import MarkerCluster


i = 2
city_name = 'berlin'
filter_name = df.index[i]
category = df['category'].iloc[i]

elements = gpd.read_file(f'../livablestreets/data/{city_name}/Features/shapes-{category}-{filter_name}.geojson')

In [61]:

map_test = folium.Map(width=750, height=400, location=[52.5205,13.3988], tiles="openstreetmap", zoom_start=14)
marker_cluster = MarkerCluster().add_to(map_test)

folium.GeoJson(mapping(elements)).add_to(marker_cluster)

loc = f'{city_name} {category} {filter_name}'
title_html = '''
             <h3 align="left" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
map_test.get_root().html.add_child(folium.Element(title_html))


map_test

In [62]:
elements = gpd.read_file(f'localfilter/{city_name}-{category}-{filter_name}.geojson')

ERROR:fiona._env:localfilter/berlin-negative-street_secondary.geojson: No such file or directory


DriverError: localfilter/berlin-negative-street_secondary.geojson: No such file or directory

In [ ]:
elements.value_counts